DO NOT RUN ALL CELLS, SOME CELLS SAVE FILES. THIS MAY OVERWRITE PREVIOUS SPLITS

In [1]:
from utils import *
import glob
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
dir_ = 'HASYv2/'
model_dir = 'trained_models/'
data_dir = 'data/'
processed_data_dir = 'processed_data/'

ORIGINAL DATASET

In [3]:
orig_dataset = read_csv(dir_+'hasy-data-labels.csv')
print(orig_dataset.head())
print('-----------------')
print(orig_dataset.info())

                     path  symbol_id latex  user_id
0  hasy-data/v2-00000.png         31     A       50
1  hasy-data/v2-00001.png         31     A       10
2  hasy-data/v2-00002.png         31     A       43
3  hasy-data/v2-00003.png         31     A       43
4  hasy-data/v2-00004.png         31     A     4435
-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168233 entries, 0 to 168232
Data columns (total 4 columns):
path         168233 non-null object
symbol_id    168233 non-null int64
latex        168233 non-null object
user_id      168233 non-null int64
dtypes: int64(2), object(2)
memory usage: 5.1+ MB
None


CUSTOM DATASET

In [4]:
# get relevant rows from dataset
up_alphas = [chr(i+65) for i in range(26)]
lw_alphas = [chr(i+97) for i in range(26)]
nums = [str(n) for n in range(10)]
ops =  ['=','+','-','\\%','*','(',')',r'\{', r'\}', r'[', r']','\\times','\\div','\\ast','\\cup','\\cap','\\subset','\\subseteq','\\supset','\\|','\\perp','<','>','\\leq','\\geq']
greeks = ['\\sigma', '\\Sigma', '\\gamma', '\\delta', '\\Delta', '\\eta', '\\theta', '\\epsilon', '\\lambda', '\\mu', '\\Pi', '\\rho', '\\phi', '\\omega', '\\ohm']
specials = ['\\$','\\&','\\#','\\infty','\\exists','\\forall','\\cdot']
relevant_labels = up_alphas + lw_alphas + nums + ops + greeks + specials
whole_dataset = orig_dataset.loc[orig_dataset['latex'].isin(relevant_labels)]

In [5]:
whole_dataset['latex'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\\sigma', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 'u', 'v', 'w', 'x', 'y', 'z', '\\Sigma',
       '\\gamma', '\\delta', '\\Delta', '\\eta', '\\theta', '\\epsilon',
       '\\lambda', '\\mu', '\\Pi', '\\rho', '\\phi', '\\omega', '\\cdot',
       '\\leq', '\\geq', '<', '>', '\\subset', '\\supset', '\\subseteq',
       '-', '+', '\\$', '\\{', '\\}', '\\&', '\\#', '\\%', '\\cup',
       '\\times', '\\ast', '\\div', '\\cap', '\\perp', '\\forall',
       '\\exists', '[', ']', '\\|', '\\infty', '\\ohm'], dtype=object)

In [6]:
# This total should be 107, = in one
len(whole_dataset['latex'].unique())

104

In [7]:
whole_dataset.head()

,path,symbol_id,latex,user_id
0,hasy-data/v2-00000.png,31,A,50
1,hasy-data/v2-00001.png,31,A,10
2,hasy-data/v2-00002.png,31,A,43
3,hasy-data/v2-00003.png,31,A,43
4,hasy-data/v2-00004.png,31,A,4435


In [8]:
ltx_dict = {'equal':'=', 'lroundb':'//(', 'rroundb':'//)'}
# function to get only alphabets from string
getalpha = lambda x: "".join([c for c in x if c.isalpha()])

# cosntruct a dataframe for custom_images
path_iter = glob.iglob("HASYv2/custom-data/*")
custom_paths = [str(p).split('/')[-1] for p in path_iter]

ltxs = [ltx_dict[getalpha(i.split('\\')[-1].split('.')[0])] for i in custom_paths]

syms = {list(ltx_dict.values())[i]:10000+i for i in range(len(ltx_dict))}
#syms ={v:i for i, v in enumerate(syms)}
symbol_ids = [syms[i] for i in ltxs]
user_ids = [7000 for i in range(len(ltxs))]

custom_dataset = pd.DataFrame(data={
    'path':custom_paths,
    'symbol_id':symbol_ids,
    'latex':ltxs,
    'user_id':user_ids 
})

# combine relevant and custom symbols
whole_dataset = pd.concat([whole_dataset, custom_dataset], axis = 0).reset_index(drop=True)
whole_dataset.drop(['user_id'], axis=1, inplace=True)
print("Number of labels in custom dataset are ", len(whole_dataset['latex'].unique()))

Number of labels in custom dataset are  107


SYMBOLS

In [9]:
# Read symbols for all classification classes
symbols = read_csv(dir_+'symbols.csv')
print(symbols.head())
print('-----------------')
print(symbols.info())

   symbol_id latex  training_samples  test_samples
0         31     A               137            22
1         32     B                53             8
2         33     C               120            14
3         34     D                50             8
4         35     E                48             6
-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 4 columns):
symbol_id           369 non-null int64
latex               369 non-null object
training_samples    369 non-null int64
test_samples        369 non-null int64
dtypes: int64(3), object(1)
memory usage: 11.7+ KB
None


In [10]:
symbols = symbols.loc[symbols['latex'].isin(relevant_labels)]
ltxs = list(ltx_dict.values())
symbol_ids = [syms[i] for i in ltxs]
train_count = {ltxs[i]:0 for i in range(len(ltxs))}
test_count = {ltxs[i]:0 for i in range(len(ltxs))}
for i in custom_paths:
    key = getalpha(i.split('\\')[-1].split('.')[0])
    train_count[ltx_dict[key]]+= 1
train_count = [train_count[i] for i in ltxs]
test_count = [test_count[i] for i in ltxs]
custom_symbols = pd.DataFrame(data={
    'symbol_id': symbol_ids,
    'latex': ltxs,
    'training_samples': train_count,
    'test_samples': test_count
})
symbols = pd.concat([symbols, custom_symbols], axis=0).reset_index(drop=True)
print("Number of lebels in symbols df is ", len(symbols))

Number of lebels in symbols df is  107


##### Reset Symbol names to start from 0 and end at max length

In [11]:
# change name of previous symbol
symbols['old_symbol'] = symbols['symbol_id']
symbols = symbols.drop(['symbol_id', 'training_samples', 'test_samples'], axis=1)

# add new id according to index of character
symbols['new_id'] = symbols.index

# make a symbols dict that references each symbol_id to the new symbol
symbols_dict = {}
for i in range(len(symbols)):
    symbols_dict[symbols['old_symbol'][i]] = symbols['new_id'][i]

In [12]:
whole_dataset['symbol_id'] = [symbols_dict[i] for i in whole_dataset['symbol_id']]
symbols_list = np.array(whole_dataset['symbol_id']).reshape(-1)

In [13]:
symbols.to_csv(processed_data_dir+'symbols.csv')

Dataset Splitting

In [14]:
# RUN THIS CELL ONLY ONCE for creating the splits, 
# then load the saved csv's for both train and test splits
train, test = get_df_split(whole_dataset, stratify_col='symbol_id', test_size=0.1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.to_csv(processed_data_dir+'train.csv')
test.to_csv(processed_data_dir+'test.csv')

In [15]:
# Use this cell to load csvs
train = read_csv(processed_data_dir+'train.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
test = read_csv(processed_data_dir+'test.csv').drop(['Unnamed: 0'], axis=1).reset_index(drop=True)

In [16]:
# (train_count, test_count, percentage of train count to total)
labels_count = get_label_count_df(train, test, symbols_list)
print(labels_count)

{0: (143, 16, 89.94), 1: (55, 6, 90.16), 2: (121, 13, 90.3), 3: (52, 6, 89.66), 4: (49, 5, 90.74), 5: (50, 6, 89.29), 6: (106, 12, 89.83), 7: (58, 6, 90.62), 8: (90, 10, 90.0), 9: (94, 10, 90.38), 10: (86, 10, 89.58), 11: (100, 11, 90.09), 12: (108, 12, 90.0), 13: (95, 10, 90.48), 14: (76, 8, 90.48), 15: (65, 7, 90.28), 16: (60, 7, 89.55), 17: (75, 8, 90.36), 18: (57, 6, 90.48), 19: (50, 6, 89.29), 20: (53, 6, 89.83), 21: (50, 6, 89.29), 22: (56, 6, 90.32), 23: (49, 5, 90.74), 24: (50, 6, 89.29), 25: (59, 6, 90.77), 26: (120, 13, 90.23), 27: (106, 12, 89.83), 28: (112, 12, 90.32), 29: (108, 12, 90.0), 30: (55, 6, 90.16), 31: (70, 8, 89.74), 32: (90, 10, 90.0), 33: (68, 7, 90.67), 34: (109, 12, 90.08), 35: (81, 9, 90.0), 36: (1011, 112, 90.03), 37: (77, 9, 89.53), 38: (51, 6, 89.47), 39: (60, 7, 89.55), 40: (52, 6, 89.66), 41: (57, 6, 90.48), 42: (59, 7, 89.39), 43: (50, 6, 89.29), 44: (52, 6, 89.66), 45: (53, 6, 89.83), 46: (48, 5, 90.57), 47: (52, 6, 89.66), 48: (50, 6, 89.29), 49: (5

One Hot Encode Labels

In [17]:
no_categories = len(symbols)
train_one_hot_symbols = convert_to_one_hot_encode(train['symbol_id'], no_categories)
test_one_hot_symbols = convert_to_one_hot_encode(test['symbol_id'], no_categories)

len of dataset 38444
len of dataset 4272


In [18]:
# whole_dataset['symbol_id_ohe'] = [list(one_hot_symbols[i]) for i in range(len(whole_dataset))]
train['symbol_id_ohe'] = [list(train_one_hot_symbols[i]) for i in range(len(train))]
test['symbol_id_ohe'] = [list(test_one_hot_symbols[i]) for i in range(len(test))]